## Aggregation Advanced

In [2]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()
restaurants = client.agg2.restaurants
airports = client.agg2.airports

## Exercise 1

In [7]:
restaurants.create_index([('address.coord', '2dsphere')])
for airport in airports.aggregate([{'$match': {'type': 'International'}}]):
    res = restaurants.aggregate([
        {'$geoNear': {
            'near': airport['loc'],
            'distanceField': 'dist',
            'maxDistance': 2000,
            'query': {'cuisine': 'Korean'}
        }}
    ])
    l_res = list(res)
    if len(l_res) == 0:
        continue
    pprint(l_res)

[{'_id': ObjectId('660bb75d4a6fa09a9fb4376d'),
  'address': {'building': '0',
              'coord': [-73.7822056, 40.6434612],
              'street': 'Jfk International Airpor',
              'zipcode': '11430'},
  'borough': 'Queens',
  'cuisine': 'Korean',
  'dist': 497.9151037865528,
  'grades': [{'date': datetime.datetime(2014, 6, 18, 0, 0),
              'grade': 'A',
              'score': 2},
             {'date': datetime.datetime(2014, 1, 14, 0, 0),
              'grade': 'A',
              'score': 12},
             {'date': datetime.datetime(2013, 1, 4, 0, 0),
              'grade': 'A',
              'score': 11},
             {'date': datetime.datetime(2012, 4, 25, 0, 0),
              'grade': 'A',
              'score': 13}],
  'name': 'Korean Lounge',
  'restaurant_id': '40625198'}]


## Exercise 2

In [13]:
metro = client.agg2.metro

res = metro.aggregate([
    {'$group': {'_id': '$line_num', 'max_pasgr_num': {'$max': '$ride_pasgr_num'}, 'min_pasgr_num': {'$min': '$ride_pasgr_num'}, 'avg_pasgr_num': {'$avg': '$ride_pasgr_num'}}},
    {'$sort': {'avg_pasgr_num': -1}}
])
pprint(list(res))

[{'_id': '2호선',
  'avg_pasgr_num': 28450.32711314476,
  'max_pasgr_num': 152285,
  'min_pasgr_num': 187},
 {'_id': '1호선',
  'avg_pasgr_num': 25665.077495840265,
  'max_pasgr_num': 115790,
  'min_pasgr_num': 2613},
 {'_id': '4호선',
  'avg_pasgr_num': 20892.54868488417,
  'max_pasgr_num': 81539,
  'min_pasgr_num': 256},
 {'_id': '경인선',
  'avg_pasgr_num': 15479.504970881864,
  'max_pasgr_num': 52930,
  'min_pasgr_num': 833},
 {'_id': '3호선',
  'avg_pasgr_num': 15471.945636888462,
  'max_pasgr_num': 98674,
  'min_pasgr_num': 1},
 {'_id': '7호선',
  'avg_pasgr_num': 13225.216640566376,
  'max_pasgr_num': 55392,
  'min_pasgr_num': 483},
 {'_id': '과천선',
  'avg_pasgr_num': 12952.302100665558,
  'max_pasgr_num': 40611,
  'min_pasgr_num': 224},
 {'_id': '경부선',
  'avg_pasgr_num': 11745.341028387787,
  'max_pasgr_num': 71684,
  'min_pasgr_num': 1},
 {'_id': '5호선',
  'avg_pasgr_num': 11062.741804835854,
  'max_pasgr_num': 103998,
  'min_pasgr_num': 314},
 {'_id': '일산선',
  'avg_pasgr_num': 10349.7383843

## Exercise 3

In [15]:
res = metro.aggregate([
    {'$match': {'use_dt': {'$gte': '20180101', '$lte': '20180107'}}},
    {'$group': {'_id': '$use_dt', 'total_alight_pasgr': {'$sum': '$alight_pasgr_num'}}},
    {'$sort': {'_id': 1}}
])
pprint(list(res))

[{'_id': '20180101', 'total_alight_pasgr': 3494792},
 {'_id': '20180102', 'total_alight_pasgr': 7477444},
 {'_id': '20180103', 'total_alight_pasgr': 7652832},
 {'_id': '20180104', 'total_alight_pasgr': 7676464},
 {'_id': '20180105', 'total_alight_pasgr': 8105183},
 {'_id': '20180106', 'total_alight_pasgr': 5984044},
 {'_id': '20180107', 'total_alight_pasgr': 4286499}]
